In [1]:
# %matplotlib inline

In [1]:
import geopandas as gpd
from shapely import Point
import pandas as pd
import h3
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np
from rasterio.enums import Resampling
from rasterio.shutil import copy as rio_copy

# Read Parquet and Convert H3 Grid to LatLng

In [2]:
# Dataset ~ https://app.hubocean.earth/catalog/dataset/5194bda1-7838-4be2-8ca8-36fa1fdfe300/
df = pd.read_parquet('5194bda1-7838-4be2-8ca8-36fa1fdfe300.parquet')

In [3]:
df = df[['S','hex6']]

In [4]:
df.head(4)

,S,hex6
0,1.0,861858c8fffffff
1,1.0,861858cb7ffffff
2,1.0,861858cd7ffffff
3,1.0,861858ce7ffffff


In [5]:
# df['geometry'] = df.hex6.apply(lambda x: Point(h3.cell_to_latlng(x)))
df['geometry'] = df.hex6.apply(lambda x: Point(h3.cell_to_latlng(x)[1], h3.cell_to_latlng(x)[0]))  # lon, lat

In [6]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [7]:
gdf = gdf[['S','geometry']]

In [8]:
print(gdf.head(4))

     S                   geometry
0  1.0   POINT (-6.40037 44.9994)
1  1.0  POINT (-6.46773 45.11078)
2  1.0  POINT (-6.36678 44.94369)
3  1.0  POINT (-6.23658 44.98175)


# Rasterize

In [9]:
gdf_m = gdf.to_crs(epsg=3857)  # Web Mercator (meters)

In [10]:
pixel_size = 20000  # meters
minx, miny, maxx, maxy = gdf_m.total_bounds
width = int((maxx - minx) / pixel_size)
height = int((maxy - miny) / pixel_size)
transform = from_origin(minx, maxy, pixel_size, pixel_size)

In [11]:
# Coordinates
xs = gdf_m.geometry.x.values
ys = gdf_m.geometry.y.values
# Pixel indices
cols = ((xs - minx) / pixel_size).astype(int)
rows = ((maxy - ys) / pixel_size).astype(int)

In [12]:
raster_sum = np.zeros((height, width), dtype=float)
raster_count = np.zeros((height, width), dtype=int)
for r, c, s in zip(rows, cols, gdf_m['S']):
    if 0 <= r < height and 0 <= c < width:
        raster_sum[r, c] += s
        raster_count[r, c] += 1

In [13]:
raster_avg = np.zeros_like(raster_sum)
mask = raster_count > 0
raster_avg[mask] = raster_sum[mask] / raster_count[mask]

In [14]:
# with rasterio.open(
#     "5194bda1-7838-4be2-8ca8-36fa1fdfe300.tif",
#     'w',
#     driver='GTiff',
#     height=height,
#     width=width,
#     count=1,
#     dtype='float32',
#     crs='EPSG:3857',
#     transform=transform,
# ) as dst:
#     dst.write(raster_avg, 1)

In [15]:
# First, write a temporary GeoTIFF
with rasterio.open(
    "5194bda1-7838-4be2-8ca8-36fa1fdfe300.tif",
    'w',
    driver='GTiff',
    height=raster_avg.shape[0],
    width=raster_avg.shape[1],
    count=1,
    dtype=raster_avg.dtype,
    crs='EPSG:3857',
    transform=transform,
    compress='deflate',   # compression
    tiled=True,           # must be tiled for COG
    blockxsize=256,       # typical tile size
    blockysize=256
) as dst:
    dst.write(raster_avg, 1)

In [16]:
# Convert to Cloud Optimized GeoTIFF
rio_copy(
    "5194bda1-7838-4be2-8ca8-36fa1fdfe300.tif",
    "5194bda1-7838-4be2-8ca8-36fa1fdfe300_cog.tif",
    copy_src_overviews=True,
    driver='COG',
    compress='deflate'
)